In [1]:
import gc
import math
import random

import torch
import torch.nn as nn

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from tqdm import trange
from pathlib import Path

d:\code\dementiakiller\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# CONFIG

# Training
LR = 0.001
MELS = 50 # FIXED due to preprocessing
BS = 32 # DO NOT CHANGE ATM fixed to numpy folder as each bach is saved as a numpy array 

DEVICE = 'cuda'

CHARSET = " .,!abcdefghijklmnopqrstuvwxyz"


 # TODO: Clean this up
FOLDER_LJ = Path('data/LJSpeech-1.1/')
FOLDER_WAV = Path('data/LJSpeech-1.1/wavs')

path_numpy = FOLDER_LJ / Path('numpy')
path_data = path_numpy / Path('data')
path_labels = path_numpy / Path('labels')
path_input = path_numpy / Path('input_lengths')
path_target = path_numpy / Path('target_lengths')

path_models = Path('models')

path_models.mkdir(exist_ok=True)

In [3]:
# Dataloader 
df = pd.read_pickle(FOLDER_LJ / 'LJSpeech-1.1.pkl')
df = df[0:500]
df.head()

,filenames,text,text2,spectrogram,encodings,input_lengths,target_lengths
0,LJ001-0001,"Printing, in the only sense with which we are ...","Printing, in the only sense with which we are ...","[[7.932281498312932e-09, 1.1178071872564033e-0...","[20, 22, 13, 18, 24, 13, 18, 11, 3, 1, 13, 18,...",1065,151
1,LJ001-0002,in being comparatively modern.,in being comparatively modern.,"[[9.356983241559647e-07, 0.0001049457350745797...","[13, 18, 1, 6, 9, 13, 18, 11, 1, 7, 19, 17, 20...",210,30
2,LJ001-0003,For although the Chinese took impressions from...,For although the Chinese took impressions from...,"[[0.00014526753511745483, 6.434485112549737e-0...","[10, 19, 22, 1, 5, 16, 24, 12, 19, 25, 11, 12,...",1066,155
3,LJ001-0004,"produced the block books, which were the immed...","produced the block books, which were the immed...","[[0.00011615546827670187, 5.416104249889031e-0...","[20, 22, 19, 8, 25, 7, 9, 8, 1, 24, 12, 9, 1, ...",567,89
4,LJ001-0005,the invention of movable metal letters in the ...,the invention of movable metal letters in the ...,"[[6.309105810942128e-05, 0.0002005760325118899...","[24, 12, 9, 1, 13, 18, 26, 9, 18, 24, 13, 19, ...",895,143


In [4]:
# Extract train and test
SPLIT = 0.8
train_index = random.sample(range(len(df)), int(SPLIT*len(df)))
test_index = list(set(list(range(len(df))))- set(train_index))
len(test_index) + len(train_index)

50

In [7]:
from torch.utils.data import DataLoader

train = DataLoader(df.iloc[train_index]['spectrogram'],batch_size=BS, num_workers=8)
test = DataLoader(df.iloc[test_index]['spectrogram'],batch_size=BS, num_workers=8)

#for i in train:
#    print(i)
#    break

2

In [ ]:
# Model

# TODO: CLEAN UP THIS!
C = 16 # BUG: Can we risk clipping for n value of C?
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.conv = nn.Sequential(
                                nn.Conv2d(1,C, kernel_size=3, stride=1, padding='same'),
                                nn.ReLU(),
                                nn.BatchNorm2d(C),
                                nn.Conv2d(C,C*2, kernel_size=3, stride=1, padding='same'),
                                nn.ReLU(),
                                nn.BatchNorm2d(C*2),
                                )                          
                                

        self.ln1 = nn.Sequential(
                                nn.Linear((C*2)*50,600),
                                nn.ReLU(),                                
                                nn.Linear(600,300),
                                nn.ReLU(),
                                )

        self.gru = nn.GRU(300, 300, batch_first=False)
        self.ln2 = nn.Sequential(
                        nn.Linear(300, 150),
                        nn.ReLU(),
                        nn.Linear(150, 75),
                        nn.ReLU(),
                        nn.Linear(75,len(CHARSET)),
                         )

    def forward(self, x):
        # TODO: Use different implementation/Input when using CONV?
        # TODO: When using conv should we change input ? Input ATM: (Time,Batch,Mel/Freq) 
        # TODO: CLEAN UP and make sur it is correct 
        x = x.reshape(x.shape[1],1,-1,MELS)
        print(x.shape)
        x = self.conv(x)
        print(x.shape)
        x = x.permute(0, 2, 1, 3)
        print("dis" + str(x.shape))
        x = x.reshape(x.shape[0],-1,(C*2)*MELS)
        x = x.permute(1,0,2)
        print(x.shape)
        x = self.ln1(x)
        print(x.shape)
        x = nn.functional.relu(self.gru(x)[0])
        print(x.shape)
        x = self.ln2(x)
        print(x.shape)
        #  Output (Time,Batch,Mel/Freq) -> Convention when using CTC loss 
        x = torch.nn.functional.log_softmax(x, dim=2)
        return x

model = Net().to(DEVICE)
model
"""

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.ln1 = nn.Sequential(
                                nn.Linear(50,100),
                                nn.ReLU(),                                
                                nn.Linear(100,250),
                                nn.ReLU(),
                                nn.Linear(250,500),
                                nn.ReLU(),
                                nn.Linear(500,500),
                                nn.ReLU(),
                                )

        self.gru = nn.GRU(500, 500, batch_first=False)
        self.ln2 = nn.Sequential(
                        nn.Linear(500, 250),
                        nn.ReLU(),
                        nn.Linear(250, 75),
                        nn.ReLU(),
                        nn.Linear(75,len(CHARSET)),
                         )

    def forward(self, x):
        x = self.ln1(x)
        x = nn.functional.relu(self.gru(x)[0])
        x = self.ln2(x)
        x = torch.nn.functional.log_softmax(x, dim=2)
        return x

model = Net().to(DEVICE)
"""

In [ ]:
# How are shapes with CTC loss working? 
# BUG: CNN We need to be more careful about shapes when using CTC loss due to tokens 
# INPUT [time,batch,mels]
inn = torch.rand(1114,32,50, device=DEVICE)
print(f"Input shape: {inn.shape}")
test = model(inn)
#
print(f"Output shape: {test.shape}")
# OUTPUT [time,batch,len(CHARSET)]

In [ ]:
# Criterion (Loss function) and Optimizer
criterion  = nn.CTCLoss(reduction='mean', zero_infinity=True)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
# TODO: we need to shuffle, currently every batch and epoch are the same
# TODO: Normalize data?
# TODO: Char mappings how are padding handled vs space 

EPOCHS = 500

traning_loss = []

t = trange(EPOCHS)
import time
for epoch in t:
    for batch in range(408):

        #start = time.time()
        batch = random.randint(0, 408) # BUG: HAX: TODO: BAD
        inputs = np.load(path_data / Path(str(batch) + '.npy'))
        labels = np.load(path_labels / Path(str(batch) + '.npy'))
        input_lengths = np.load(path_input / Path(str(batch) + '.npy'))
        target_lengths = np.load(path_target / Path(str(batch) + '.npy'))

        inputs = torch.tensor(inputs).reshape(-1,BS,50).float().to(DEVICE)
        labels = torch.tensor(labels).float().to(DEVICE)
        input_lengths = torch.tensor(input_lengths)
        target_lengths = torch.tensor(target_lengths)
        
        # NOTE: 10s to load this way per epoch for LJSpeech
        #end = time.time()
        #print(end-start)
        
        #inputs = torch.nn.functional.normalize(inputs)

        optimizer.zero_grad() # Reset gradient to zero

        # Forward
        output = model(inputs) 

        # Compute loss
        loss = criterion(output, labels, input_lengths,target_lengths)

        # Update loss
        loss.backward()

        # optimizer
        optimizer.step()
        if batch % 10 == 0:
            t.set_description(f"Loss: {loss.item()}")

        del inputs
        del labels
    
    if True:
        torch.save(model.state_dict(), f"models/{str(epoch)}-c.pt")
    
    traning_loss.append(loss.item())
    


In [ ]:
plt.plot(traning_loss)

In [ ]:
# TEST
TEST = 23 
TEST2 = 0

data = np.load(path_data / Path(str(TEST) + '.npy'))
data = torch.tensor(data[TEST2]).float()
data = data[:,:,None].permute(1,2,0).to(DEVICE)
print(data.shape)

# Run the model
output = model(data)
output = output[:, 0, :].argmax(dim=1)

estimate = []
for char in output:
    if char != 0:
        estimate.append(CHARSET[char-1])
print(''.join(estimate))